# Linking via Upper Ontology

## TERMite in action

## Drug Active Substance

In [1]:
import spark_session_builder

sparkSession, sc = spark_session_builder.session()
sc

<SparkContext master=local[*] appName=testing>

The linking process generates 3 topics
- Resnet to Reaxys
- PP to Reaxys
- PP to Resnet

In [2]:
hdfs_path = "hdfs://demo-full-load-hdfs-namenode:8020"
r2rx_data_path = hdfs_path + "/links/resnet-reaxys"
pp2rx_data_path = hdfs_path + "/links/pp-reaxys"
pp2rs_data_path = hdfs_path + "/links/pp-resnet"
r2rx_filename = r2rx_data_path + "/*.csv"
pp2rx_filename = pp2rx_data_path + "/*.csv"
pp2rs_filename = pp2rs_data_path + "/*.csv"

In [3]:
from IPython.core.display import HTML


def show_top(df, records=10):
    """
    display the top n records in a table
    """
    display(HTML(df.limit(records).toPandas().to_html(escape=False)))

## Resnet to Reaxys

In [4]:
r2rx_df = sparkSession \
    .read \
    .option("header", "true") \
    .csv(r2rx_filename)
show_top(r2rx_df)

,hdfsIngestTime,offset,fingerprint,reaxys_chemicalcompoundname,resnet_smallmol,provenanceId
0,2021-02-24T11:55:24.301359,95505,iri:uniprot/CHEMBL561481,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/3138034,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2733435,None
1,2021-02-24T11:55:24.301359,95505,iri:uniprot/CHEMBL561481,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2733435,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/3138034,None
2,2021-02-24T11:55:24.301369,95506,iri:uniprot/CHEMBL561481,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2733435,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/3138034,None
3,2021-02-24T11:55:24.301369,95506,iri:uniprot/CHEMBL561481,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/3138034,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2733435,None
4,2021-02-24T11:55:24.304186,150196,iri:uniprot/CHEMBL42228,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/6164356,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038181225,None
5,2021-02-24T11:55:24.305224,155785,iri:uniprot/CHEMBL276334,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/774966,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038182021,None
6,2021-02-24T11:55:24.304651,156616,iri:uniprot/CHEMBL2323854,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/3195859,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038199220,None
7,2021-02-24T11:55:24.307341,156647,iri:uniprot/CHEMBL3337531,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2041500,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038113731,None
8,2021-02-24T11:55:24.307588,157382,iri:uniprot/CHEMBL416578,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/24032549,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038094213,None
9,2021-02-24T11:55:24.308715,157510,iri:uniprot/CHEMBL2108765,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/11465497,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038135438,None


## Pharmapendium to Reaxys

In [5]:
pp2rx_df = sparkSession \
    .read \
    .option("header", "true") \
    .csv(pp2rx_filename)
show_top(pp2rx_df)

,hdfsIngestTime,offset,fingerprint,ppplus_drug,reaxys_chemicalcompoundname,provenanceId
0,2021-02-24T11:55:22.99909,24097,iri:uniprot/CHEMBL3218576,https://data.elsevier.com/lifescience/entity/ppplus/drug/jO4qcQ5aUr9,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038135647,None
1,2021-02-24T11:55:23.004095,24098,iri:uniprot/CHEMBL3218576,https://data.elsevier.com/lifescience/entity/ppplus/drug/jO4qcQ5aUr9,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038135647,None
2,2021-02-24T11:55:23.010593,28542,iri:uniprot/CHEMBL841,https://data.elsevier.com/lifescience/entity/ppplus/drug/5G3oKDOTrf_,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038102779,None
3,2021-02-24T11:55:23.01143,28543,iri:uniprot/CHEMBL841,https://data.elsevier.com/lifescience/entity/ppplus/drug/5G3oKDOTrf_,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038102779,None
4,2021-02-24T11:55:23.051009,34198,iri:uniprot/CHEMBL170797,https://data.elsevier.com/lifescience/entity/ppplus/drug/LmZQWquqxuF,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038125122,None
5,2021-02-24T11:55:23.054978,34199,iri:uniprot/CHEMBL170797,https://data.elsevier.com/lifescience/entity/ppplus/drug/LmZQWquqxuF,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038125122,None
6,2021-02-24T11:55:23.057858,35278,iri:uniprot/CHEMBL486174,https://data.elsevier.com/lifescience/entity/ppplus/drug/FNJ8ldAiEOA,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038103204,None
7,2021-02-24T11:55:23.05872,35279,iri:uniprot/CHEMBL486174,https://data.elsevier.com/lifescience/entity/ppplus/drug/FNJ8ldAiEOA,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038103204,None
8,2021-02-24T11:55:23.060412,35283,iri:uniprot/CHEMBL1201213,https://data.elsevier.com/lifescience/entity/ppplus/drug/mBTPxofGut7,https://data.elsevier.com/lifescience/entity/ppplus/drug/ADA-62DMlw9,None
9,2021-02-24T11:55:23.060412,35283,iri:uniprot/CHEMBL1201213,https://data.elsevier.com/lifescience/entity/ppplus/drug/ADA-62DMlw9,https://data.elsevier.com/lifescience/entity/ppplus/drug/mBTPxofGut7,None


## Pharmapendium to Resnet

In [6]:
pp2rs_df = sparkSession \
    .read \
    .option("header", "true") \
    .csv(pp2rs_filename)
show_top(pp2rs_df)

,hdfsIngestTime,offset,fingerprint,ppplus_drug,resnet_smallmol,provenanceId
0,2021-02-24T11:55:24.559873,45395,iri:uniprot/CHEMBL1201346,https://data.elsevier.com/lifescience/entity/ppplus/drug/DH4u_TrfagG,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038093101,None
1,2021-02-24T11:55:24.559863,45512,iri:uniprot/CHEMBL2043437,https://data.elsevier.com/lifescience/entity/ppplus/drug/j9sCxvbs8NA,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038136039,None
2,2021-02-24T11:55:24.568017,45514,iri:uniprot/CHEMBL516,https://data.elsevier.com/lifescience/entity/ppplus/drug/aZ1m_BKrhL8,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038076869,None
3,2021-02-24T11:55:24.567722,45546,iri:uniprot/CHEMBL312448,https://data.elsevier.com/lifescience/entity/ppplus/drug/3LL9lyH0tN-,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099059,None
4,2021-02-24T11:55:24.572663,45606,iri:uniprot/CHEMBL631,https://data.elsevier.com/lifescience/entity/ppplus/drug/gTokRyMybs4,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038088761,None
5,2021-02-24T11:55:24.572248,45752,iri:uniprot/CHEMBL1201471,https://data.elsevier.com/lifescience/entity/ppplus/drug/qo6FO7aGFJF,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038096035,None
6,2021-02-24T11:55:24.575978,45754,iri:uniprot/CHEMBL1200709,https://data.elsevier.com/lifescience/entity/ppplus/drug/P9DmldiDXr9,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038095378,None
7,2021-02-24T11:55:24.575333,45787,iri:uniprot/CHEMBL3137342,https://data.elsevier.com/lifescience/entity/ppplus/drug/LY7jJYb6ia-,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038135495,None
8,2021-02-24T11:55:24.579929,45798,iri:uniprot/CHEMBL2107448,https://data.elsevier.com/lifescience/entity/ppplus/drug/MZuYA_lTIz9,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038095367,None
9,2021-02-24T11:55:24.578934,46186,iri:uniprot/CHEMBL2108638,https://data.elsevier.com/lifescience/entity/ppplus/drug/Yg5ymXArLR6,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038135105,None


# Building a single view of Drugs

In [7]:
from PWBdata import PWBdata
pwb = PWBdata(hdfs_path)
resnet_Smallmol_df = pwb.get_dataframe(sparkSession, 'resnet', 'Smallmol')

In [8]:
r2rx_with_resnet_names = r2rx_df.join(resnet_Smallmol_df, r2rx_df.resnet_smallmol == resnet_Smallmol_df.SmallMol).select('fingerprint','reaxys_chemicalcompoundname', 'resnet_smallmol', 'hasName').withColumnRenamed('hasName', 'resnet_name')

In [9]:
show_top(r2rx_with_resnet_names)

,fingerprint,reaxys_chemicalcompoundname,resnet_smallmol,resnet_name
0,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/20487708,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate
1,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/30985940,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate
2,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/15494998,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate
3,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/34095599,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate
4,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/32058082,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate
5,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/8101020,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate
6,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/19732180,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate
7,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/29555266,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate
8,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/8003908,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate
9,iri:uniprot/CHEMBL1229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/19882123,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038079849,oseltamivir carboxylate


In [10]:
rmc_compoundname_df = pwb.get_dataframe(sparkSession, 'rmc', 'chemicalcompoundname').select('ChemicalCompoundName','hasChemicalCompound', 'hasName')

In [11]:
r2rx_with_resnet_and_RMC_names = r2rx_with_resnet_names.join(rmc_compoundname_df, rmc_compoundname_df.hasChemicalCompound == r2rx_with_resnet_names.reaxys_chemicalcompoundname) \
    .withColumnRenamed('hasName', 'reaxys_name').withColumnRenamed('reaxys_chemicalcompoundname', 'reaxys_chemicalcompound').drop('hasChemicalCompound') \
    .withColumnRenamed('ChemicalCompoundName', 'reaxys_chemicalcompoundname').dropDuplicates()
show_top(r2rx_with_resnet_and_RMC_names)

,fingerprint,reaxys_chemicalcompound,resnet_smallmol,resnet_name,reaxys_chemicalcompoundname,reaxys_name
0,iri:uniprot/CHEMBL3990044,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/30557944,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038217733,relatlimab,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1741800440,Medarex/BMS
1,iri:uniprot/CHEMBL275707,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/6457432,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038092805,rimcazole,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/197064520,rimcazole dihydrochloride
2,iri:uniprot/CHEMBL300049,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/11975279,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038204809,ABT-229,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/171785131,ABT-229
3,iri:uniprot/CHEMBL1201239,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2175358,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038093807,iodamide,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1746489239,Iodamide
4,iri:uniprot/CHEMBL511,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/269019,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038091995,pyrilamine maleate,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1871888849,[3H]-mepyramine
5,iri:uniprot/CHEMBL1159684,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2812196,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038175533,menadione sodium bisulfite,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1472501908,menadione bisulfite
6,iri:uniprot/CHEMBL71,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/3779989,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038092737,chlorpromazine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1825128031,chlorpromazine chloride
7,iri:uniprot/CHEMBL71,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/3779989,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038104840,chlorazine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1755464278,CPZ
8,iri:uniprot/CHEMBL70,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/4711761,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038145221,Embeda,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/10371549,morphine
9,iri:uniprot/CHEMBL1213490,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/6854221,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038104534,FK228,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/10943094,(E)-(1S 4S 10S 21R)-7-[(Z)-ethylideno]-4 21-diisopropyl-2-oxa-12 13-dithia-5 8 20 23-tetraazabicyclo[8.7.6]tricos-16-ene-3 6 9 19 22-pentaone


In [12]:
pp_drug_df = pwb.get_dataframe(sparkSession, 'pp', 'Drug').select('Drug','hasName', 'hasCASs', 'hasChemicalNames')
show_top(pp_drug_df)

,Drug,hasName,hasCASs,hasChemicalNames
0,https://data.elsevier.com/lifescience/entity/ppplus/drug/021oG0dA6pC,Dextroamphetamine; Pyrilamine Maleate,None,None
1,https://data.elsevier.com/lifescience/entity/ppplus/drug/152tElrWGOA,Ritonavir,155213-67-5,2 4 7 12-TETRAAZATRIDECAN-13-OIC ACID 10-HYDROXY-2-METHYL-5-(1-METHYLETHYL)-1-(2-(1-METHYLETHYL)-4-THIAZOLYL)-3 6-DIOXO-8 11-BIS(PHENYLMETHYL)- 5-THIAZOLYLMETHYL ESTER (5S-(5R* 8R* 10R* 11R*))-
2,https://data.elsevier.com/lifescience/entity/ppplus/drug/3ZWcWL7ltf_,Bosentan,147536-97-8,BENZENESULFONAMIDE 4-(1 1-DIMETHYLETHYL)-N-(6-(2-HYDROXYETHOXY)-5-(2-METHOXYPHENOXY)(2 2'-BIPYRIMIDIN)-4-YL)-
3,https://data.elsevier.com/lifescience/entity/ppplus/drug/7zryhdNTlD9,Methadone Hydrochloride; Naloxone Hydrochloride,None,None
4,https://data.elsevier.com/lifescience/entity/ppplus/drug/9FfwKbKzhUD,Ambuphylline; Doxylamine Succinate; Etafedrine Hydrochloride,None,None
5,https://data.elsevier.com/lifescience/entity/ppplus/drug/AmRxO4CRzY_,Sucroferric Oxyhydroxide,12134-57-5,POLYNUCLEAR IRON(III) OXYHYDROXIDE STABILIZED WITH SUCROSE AND STARCHES
6,https://data.elsevier.com/lifescience/entity/ppplus/drug/Db0fozDTVu9,Insulin Susp Protamine Zinc Purified Beef,None,None
7,https://data.elsevier.com/lifescience/entity/ppplus/drug/Ddp7U1pEaiG,Rapacuronium Bromide,156137-99-4,PIPERIDINIUM 1-((2-BETA 3-ALPHA 5-ALPHA 16-BETA 17-BETA)-3-(ACETYLOXY)-17-(1-OXOPROPOXY)-2-(1-PIPERIDINYL)ANDROSTAN-16-YL)-1-(2-PROPENYL)- BROMIDE
8,https://data.elsevier.com/lifescience/entity/ppplus/drug/FkXLXkslE69,Tolnaftate,2398-96-1,CARBANILIC ACID M N-DIMETHYLTHIO- O-2-NAPHTHYL ESTER
9,https://data.elsevier.com/lifescience/entity/ppplus/drug/H_Pm1Bx2SiB,Ketoconazole,65277-42-1,PIPERAZINE 1-ACETYL-4-(4-((2-(2 4-DICHLOROPHENYL)-2-(1H-IMIDAZOL-1-YLMETHYL)-1 3-DIOXOLAN-4-YL)METHOXY)PHENYL)- CIS-


In [13]:
pp_drug_with_names = pp2rx_df.join(pp_drug_df, pp_drug_df.Drug == pp2rx_df.ppplus_drug).select('fingerprint', 'ppplus_drug', 'hasName').withColumnRenamed('hasName','ppname').dropDuplicates()
show_top(pp_drug_with_names)

,fingerprint,ppplus_drug,ppname
0,iri:uniprot/CHEMBL2105131,https://data.elsevier.com/lifescience/entity/ppplus/drug/DX4gf5x4BOA,Mefruside
1,iri:uniprot/CHEMBL1819440,https://data.elsevier.com/lifescience/entity/ppplus/drug/FW6X4igjx7H,Ornipressin
2,iri:uniprot/CHEMBL2103837,https://data.elsevier.com/lifescience/entity/ppplus/drug/wK3REbwdjzE,Tafamidis Meglumine
3,iri:uniprot/CHEMBL3991241,https://data.elsevier.com/lifescience/entity/ppplus/drug/MKapj5TY4eG,Ferric Citrate
4,iri:uniprot/CHEMBL669,https://data.elsevier.com/lifescience/entity/ppplus/drug/n5ZQ6Sd1dNA,Cyclobenzaprine Hydrochloride
5,iri:uniprot/CHEMBL477772,https://data.elsevier.com/lifescience/entity/ppplus/drug/b-M469_u84C,Pazopanib Hydrochloride
6,iri:uniprot/CHEMBL3,https://data.elsevier.com/lifescience/entity/ppplus/drug/hPPKQ_H18U8,Nicotine Polacrilex
7,iri:uniprot/CHEMBL277522,https://data.elsevier.com/lifescience/entity/ppplus/drug/SZrNPSXptf_,Fenbufen
8,iri:uniprot/CHEMBL376140,https://data.elsevier.com/lifescience/entity/ppplus/drug/prlVuqaUPF8,Tigecycline
9,iri:uniprot/CHEMBL888,https://data.elsevier.com/lifescience/entity/ppplus/drug/rhng-h1h8C_,Gemcitabine Hydrochloride


In [14]:
drug = r2rx_with_resnet_and_RMC_names.join(pp_drug_with_names, 'fingerprint')
show_top(drug, 20)

,fingerprint,reaxys_chemicalcompound,resnet_smallmol,resnet_name,reaxys_chemicalcompoundname,reaxys_name,ppplus_drug,ppname
0,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/11094627,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/258323251,colchicine,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine
1,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2792699,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/40608870,(S)-7-acetylamino-1 2 3 10-tetramethoxy-6 7-dihydro-5H-[10-14C]benzo[a]heptalen-9-one,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine
2,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/6359065,Dimoxyliniumphosphat,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine
3,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/6359063,1-(4-Aethoxy-3-methoxy-benzyl)-6 7-dimethoxy-3-methyl-isochinolin,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine
4,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/28905385,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1596839562,colchicine,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine
5,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1959005117,vasodilator,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine
6,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1824764420,colchicine,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine
7,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/6359064,1-(4-ethoxy-3-methoxy-benzyl)-6 7-dimethoxy-3-methyl-isoquinoline,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine
8,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2792699,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/40608871,Colchicin,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine
9,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/28287847,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,colchicine,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompoundname/1550780236,colchicine,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,Colchicine


## Full Outer Join

In [18]:
resnet2reaxys_df = r2rx_df.select('fingerprint','reaxys_chemicalcompoundname','resnet_smallmol').withColumnRenamed('reaxys_chemicalcompoundname', 'reaxys_chemicalcompound')
pp2reaxys_df = pp2rx_df.select('fingerprint', 'reaxys_chemicalcompoundname', 'ppplus_drug').withColumnRenamed('ppplus_drug', 'pharmapendium_drug').drop('reaxys_chemicalcompoundname')
combined = resnet2reaxys_df.join(pp2reaxys_df, 'fingerprint', how='fullouter')
show_top(combined)

,fingerprint,reaxys_chemicalcompound,resnet_smallmol,pharmapendium_drug
0,iri:uniprot/CHEMBL100940,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/4359362,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038183050,None
1,iri:uniprot/CHEMBL100940,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/34422449,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038183050,None
2,iri:uniprot/CHEMBL100940,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/4359362,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038183050,None
3,iri:uniprot/CHEMBL100940,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/34422449,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038183050,None
4,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9
5,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9
6,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9
7,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9
8,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9
9,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9


In [26]:
from pyspark.sql.functions import substring_index, regexp_replace
df_for_match = combined.withColumn('CHEMBL_ID', regexp_replace(substring_index(combined.fingerprint, '/', -1)," ", ""))
show_top(df_for_match)

,fingerprint,reaxys_chemicalcompound,resnet_smallmol,pharmapendium_drug,CHEMBL_ID
0,iri:uniprot/CHEMBL100940,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/4359362,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038183050,None,CHEMBL100940
1,iri:uniprot/CHEMBL100940,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/34422449,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038183050,None,CHEMBL100940
2,iri:uniprot/CHEMBL100940,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/4359362,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038183050,None,CHEMBL100940
3,iri:uniprot/CHEMBL100940,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/34422449,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038183050,None,CHEMBL100940
4,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,CHEMBL107
5,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,CHEMBL107
6,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,CHEMBL107
7,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,CHEMBL107
8,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,CHEMBL107
9,iri:uniprot/CHEMBL107,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/45063,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038099544,https://data.elsevier.com/lifescience/entity/ppplus/drug/mbbDetVXUr9,CHEMBL107


In [27]:
chembl_compound_record_df = pwb.get_dataframe(sparkSession, 'chembl', 'Compound').select('Compound','hasChemblId', 'hasPreferredMoleculeName') \
    .withColumnRenamed('hasChemblId', 'CHEMBL_ID')
show_top(chembl_compound_record_df, 2)

,Compound,CHEMBL_ID,hasPreferredMoleculeName
0,https://data.elsevier.com/lifescience/entity/chembl/compound/1000522,CHEMBL1576638,None
1,https://data.elsevier.com/lifescience/entity/chembl/compound/1001413,CHEMBL1577529,None


In [29]:
compound_entities = df_for_match.join(chembl_compound_record_df, 'CHEMBL_ID' ).dropDuplicates()
show_top(compound_entities)

,CHEMBL_ID,fingerprint,reaxys_chemicalcompound,resnet_smallmol,pharmapendium_drug,Compound,hasPreferredMoleculeName
0,CHEMBL1098658,iri:uniprot/CHEMBL1098658,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/1315490,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038096615,None,https://data.elsevier.com/lifescience/entity/chembl/compound/627416,YANGONIN
1,CHEMBL1098658,iri:uniprot/CHEMBL1098658,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/6727980,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038096615,None,https://data.elsevier.com/lifescience/entity/chembl/compound/627416,YANGONIN
2,CHEMBL1098658,iri:uniprot/CHEMBL1098658,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/230710,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038096615,None,https://data.elsevier.com/lifescience/entity/chembl/compound/627416,YANGONIN
3,CHEMBL1098658,iri:uniprot/CHEMBL1098658,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/5054110,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038096615,None,https://data.elsevier.com/lifescience/entity/chembl/compound/627416,YANGONIN
4,CHEMBL1195,iri:uniprot/CHEMBL1195,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/2382110,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038195017,https://data.elsevier.com/lifescience/entity/ppplus/drug/KGYF1M-OomG,https://data.elsevier.com/lifescience/entity/chembl/compound/208978,PROPOXYCAINE
5,CHEMBL1200691,iri:uniprot/CHEMBL1200691,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/3692530,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038091264,None,https://data.elsevier.com/lifescience/entity/chembl/compound/674642,MAGNESIUM ACETATE
6,CHEMBL1200691,iri:uniprot/CHEMBL1200691,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/6855415,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038091264,None,https://data.elsevier.com/lifescience/entity/chembl/compound/674642,MAGNESIUM ACETATE
7,CHEMBL1200691,iri:uniprot/CHEMBL1200691,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/33175380,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038091264,None,https://data.elsevier.com/lifescience/entity/chembl/compound/674642,MAGNESIUM ACETATE
8,CHEMBL1231,iri:uniprot/CHEMBL1231,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/29761378,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038118783,https://data.elsevier.com/lifescience/entity/ppplus/drug/QoN5wrZ_219,https://data.elsevier.com/lifescience/entity/chembl/compound/229058,OXYBUTYNIN
9,CHEMBL1231,iri:uniprot/CHEMBL1231,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/29761378,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038118783,https://data.elsevier.com/lifescience/entity/ppplus/drug/Q1MrjjfKZN9,https://data.elsevier.com/lifescience/entity/chembl/compound/229058,OXYBUTYNIN
